In [1]:
import torch 
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel, ViTConfig
from PIL import Image
import requests
import matplotlib.pyplot as plt

import numpy as np
import cv2
from datasets import load_dataset,load_metric
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

from accelerate import Accelerator

accelerator = Accelerator()
device = accelerator.device


/opt/crc/c/conda/23.5.2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [2]:
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# url = "http://farm3.staticflickr.com/2066/1798910782_5536af8767_z.jpg"
# url = "http://farm1.staticflickr.com/184/399924547_98e6cef97a_z.jpg"
url = "http://farm1.staticflickr.com/128/318959350_1a39aae18c_z.jpg"
image = Image.open(requests.get(url, stream=True).raw)

pretrained_name = 'google/vit-base-patch16-224'
# pretrained_name = 'vit-base-patch16-224-finetuned-imageneteval'
# pretrained_name = 'openai/clip-vit-base-patch32'
config = ViTConfig.from_pretrained(pretrained_name)
processor = ViTImageProcessor.from_pretrained(pretrained_name)
# get mean and std to unnormalize the processed images
mean, std = processor.image_mean, processor.image_std

pred_model = ViTForImageClassification.from_pretrained(pretrained_name)
pred_model.to(device)
# set to eval mode
pred_model.eval()

with torch.no_grad():
    inputs = processor(images=image, return_tensors="pt")
    inputs.to(device)
    outputs = pred_model(**inputs, output_hidden_states=True)
    logits = outputs.logits
    # model predicts one of the 1000 ImageNet classes
    predicted_class_idx = logits.argmax(-1).item()
    print("Predicted class:", pred_model.config.id2label[predicted_class_idx])

Predicted class: golden retriever


In [3]:
from maskgen.models.mask_generating_model8 import MaskGeneratingModel

mask_gen_model = MaskGeneratingModel(pred_model, hidden_size=config.hidden_size, num_classes=config.num_labels)
mask_gen_model.to(device)
print()

# training

In [4]:
from torch.utils.data import DataLoader
def load_data(seed=42): 
    dataset = load_dataset("mrm8488/ImageNet1K-val")
    dataset = dataset['train']
    splits = dataset.train_test_split(test_size=0.1, seed=seed)
    test_ds = splits['test']
    splits = splits['train'].train_test_split(test_size=0.1, seed=seed)
    train_ds = splits['train']
    val_ds = splits['test']
    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = load_data()

normalize = Normalize(mean=processor.image_mean, std=processor.image_std)
if "height" in processor.size:
    size = (processor.size["height"], processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in processor.size:
    size = processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = processor.size.get("longest_edge")

transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

def preprocess(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

train_ds.set_transform(preprocess)


Repo card metadata block was not found. Setting CardData to empty.


# training params

In [5]:
batch_size = 256
num_workers = 8
train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=num_workers)
n_steps = 10

params_to_optimize = [name for name, param in mask_gen_model.named_parameters() if param.requires_grad]
print("params to be optimized: ")
print(params_to_optimize)

params to be optimized: 
['similarity_measure.logit_scale', 'similarity_measure.query.weight', 'similarity_measure.query.bias', 'similarity_measure.key.weight', 'similarity_measure.key.bias']


In [6]:
from tqdm import tqdm

params_to_optimize = [param for param in mask_gen_model.parameters() if param.requires_grad]
# optimizer = torch.optim.Adam(params_to_optimize, lr=1e-3, weight_decay=1e-5)
optimizer = torch.optim.Adam(params_to_optimize, lr=1e-3, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.9)


print()

for epoch in range(10):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        pixel_values = data['pixel_values'].to(device)
        loss_dict = mask_gen_model.train_one_batch(pixel_values, optimizer=optimizer, n_steps=n_steps)
        # scheduler.step()
        pbar.set_description(f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss_dict['loss'].item():.4f}, " 
                             f"Reward Loss = {loss_dict['reward_loss'].item():.4f}, "
                            #  f"Regret Loss = {loss_dict['regret_loss'].item():.4f}, "
                             f"Mask Loss = {loss_dict['mask_loss'].item():.4f} "
                             f"alt_mask_loss = {loss_dict['alt_mask_loss'].item():.4f}"
                             f"mask_mean = {loss_dict['mask_mean'].item():.4f} "
                             f"prob_mean = {loss_dict['prob_mean'].item():.4f} "
                             )
        if idx % 10 == 0:
            print()
        if (idx) % 100 == 0:
            
            torch.save(mask_gen_model.state_dict(), f'mask_gen_model/mask_gen_model_{epoch}_{idx}.pth') 



torch.save(mask_gen_model.state_dict(), f'mask_gen_model/mask_gen_model_final_{epoch}_{idx}.pth') 


Epoch 1, Step 1: Loss = 0.4750, Reward Loss = 0.3881, Mask Loss = 0.0870 alt_mask_loss = 0.8601mask_mean = 0.4998 prob_mean = 0.5613 :   0%|          | 0/159 [00:06<?, ?it/s]

Epoch 1, Step 11: Loss = 0.1321, Reward Loss = 0.1205, Mask Loss = 0.0192 alt_mask_loss = 0.1144mask_mean = 0.1940 prob_mean = 0.2280 :   7%|▋         | 11/159 [00:58<11:37,  4.72s/it]

Epoch 1, Step 21: Loss = 0.1068, Reward Loss = 0.0986, Mask Loss = 0.0143 alt_mask_loss = 0.0798mask_mean = 0.1652 prob_mean = 0.1885 :  13%|█▎        | 21/159 [01:44<10:43,  4.67s/it]

Epoch 1, Step 31: Loss = 0.1307, Reward Loss = 0.1184, Mask Loss = 0.0218 alt_mask_loss = 0.1209mask_mean = 0.2041 prob_mean = 0.2250 :  19%|█▉        | 31/159 [02:31<09:55,  4.65s/it]

Epoch 1, Step 41: Loss = 0.1145, Reward Loss = 0.1059, Mask Loss = 0.0168 alt_mask_loss = 0.0839mask_mean = 0.1756 prob_mean = 0.2035 :  26%|██▌       | 41/159 [03:17<09:06,  4.63s/it]

...

Epoch 3, Step 51: Loss = 0.1373, Reward Loss = 0.1249, Mask Loss = 0.0209 alt_mask_loss = 0.1223mask_mean = 0.1808 prob_mean = 0.2426 :  32%|███▏      | 51/159 [04:03<08:22,  4.65s/it]

Epoch 3, Step 61: Loss = 0.1135, Reward Loss = 0.1042, Mask Loss = 0.0173 alt_mask_loss = 0.0915mask_mean = 0.1814 prob_mean = 0.2025 :  38%|███▊      | 61/159 [04:50<07:35,  4.65s/it]

Epoch 3, Step 71: Loss = 0.1108, Reward Loss = 0.1012, Mask Loss = 0.0176 alt_mask_loss = 0.0947mask_mean = 0.1846 prob_mean = 0.1976 :  45%|████▍     | 71/159 [05:36<06:49,  4.65s/it]

Epoch 3, Step 81: Loss = 0.1263, Reward Loss = 0.1146, Mask Loss = 0.0201 alt_mask_loss = 0.1158mask_mean = 0.1866 prob_mean = 0.2231 :  51%|█████     | 81/159 [06:23<06:03,  4.66s/it]

Epoch 3, Step 88: Loss = 0.1339, Reward Loss = 0.1217, Mask Loss = 0.0201 alt_mask_loss = 0.1197mask_mean = 0.1801 prob_mean = 0.2355 :  55%|█████▌    | 88/159 [06:59<06:37,  5.60s/it]

In [ ]:
3262

3262